In [1]:
import os
import shutil
import glob
from tqdm import tqdm

In [3]:
Raw_DIR= r'C:\Users\Admin\Documents\CS583_project\mrlEyes_2018_01\mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(Raw_DIR):
    for i in tqdm([f for f in filenames if f.endswith('.png')]):
        if i.split('_')[4]=='0':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Admin\Documents\CS583_project\prepared_data\closed eye')
        
        elif i.split('_')[4]=='1':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Admin\Documents\CS583_project\prepared_data\open eye')


0it [00:00, ?it/s]
100%|███████████████████████████████████████| 10257/10257 [01:51<00:00, 92.05it/s]


# model training


In [5]:
pip install tensorflow

  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\google\\~upb\\_message.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [6]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
batchsize=8

In [5]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 50940 images belonging to 2 classes.
Found 12733 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 21225 images belonging to 2 classes.


In [7]:
bmodel = MobileNet(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

17225924/17225924 [==============================] - 14s 1us/step


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 40, 40, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 40, 40, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 40, 40, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 40, 40, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 40, 40, 32)       128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 5, 5, 512)        2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 5, 5, 512)         0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 5, 5, 512)        4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 5, 5, 512)        2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 5, 5, 512)         0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 5, 5, 512)         262144    
                                                                 
 conv_pw_9

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'C:\Users\Admin\Documents\CS583_project\models\mobilenet.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11900\3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
6367/6367 [==============================] - ETA: 0s - loss: 0.2019 - accuracy: 0.9247
Epoch 1: val_loss improved from inf to 0.30390, saving model to C:\Users\Admin\Documents\CS583_project\models\mobilenet.h5
6367/6367 [==============================] - 797s 124ms/step - loss: 0.2019 - accuracy: 0.9247 - val_loss: 0.3039 - val_accuracy: 0.8796 - lr: 0.0010
Epoch 2/5
6367/6367 [==============================] - ETA: 0s - loss: 0.1881 - accuracy: 0.9292
Epoch 2: val_loss did not improve from 0.30390
6367/6367 [==============================] - 459s 72ms/step - loss: 0.1881 - accuracy: 0.9292 - val_loss: 0.3162 - val_accuracy: 0.8848 - lr: 0.0010
Epoch 3/5
6366/6367 [============================>.] - ETA: 0s - loss: 0.1801 - accuracy: 0.9327
Epoch 3: val_loss did not improve from 0.30390
6367/6367 [==============================] - 453s 71ms/step - loss: 0.1801 - accuracy: 0.9327 - val_loss: 0.3476 - val_accuracy: 0.8687 - lr: 0.0010
Epoch 4/5
6367/6367 [=======================

In [13]:
loss_tr, acc_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11900\515593128.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_tr, acc_tr = model.evaluate_generator(train_data)


0.9478013515472412
0.13989564776420593


In [14]:
loss_vr, acc_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11900\2909778475.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_vr, acc_vr = model.evaluate_generator(validation_data)


0.885729968547821
0.31483522057533264


# model 2 

In [15]:
train_datagen_small= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data_small= train_datagen_small.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data_small= train_datagen_small.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 50940 images belonging to 2 classes.
Found 12733 images belonging to 2 classes.


In [16]:
test_datagen_small = ImageDataGenerator(rescale=1./255)

test_data_small = test_datagen.flow_from_directory(r'C:\Users\Admin\Downloads\mrl\mrl\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 21225 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

# Create the base EfficientNetB0 model
mobile_base = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))

# Build your custom model on top of MobileNetV2
custom_top = mobile_base.output
custom_top = Flatten()(custom_top)
custom_top = Dense(64, activation='relu')(custom_top)
custom_top = Dropout(0.5)(custom_top)
custom_top = Dense(2, activation='softmax')(custom_top)

# Combine the base model with your custom layers
custom_model = Model(inputs=mobile_base.input, outputs=custom_top)

# Freeze the layers of the base MobileNetV2 model
for layer in mobile_base.layers:
    layer.trainable = False


In [18]:
custom_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 80, 80, 3)    0           ['input_2[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 80, 80, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 80, 80, 3)    0           ['normalization[0][0]']          
                                                                                            

                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 20, 20, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 20, 20, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, 20, 20, 144)  1296       ['block2b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2b_b

 block3b_dwconv (DepthwiseConv2  (None, 10, 10, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 10, 10, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3b_activation (Activation  (None, 10, 10, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3b_se_squeeze (GlobalAver  (None, 240)         0           ['block3b_activation[0][0]']     
 agePoolin

 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block4b_se_reduce[0][0]']      
          

 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
                                                                                                  
 block5a_se_excite (Multiply)   (None, 5, 5, 480)    0           ['block5a_activation[0][0]',     
                                                                  'block5a_se_expand[0][0]']      
                                                                                                  
 block5a_project_conv (Conv2D)  (None, 5, 5, 112)    53760       ['block5a_se_excite[0][0]']      
          

                                                                                                  
 block5c_project_bn (BatchNorma  (None, 5, 5, 112)   448         ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 5, 5, 112)    0           ['block5c_project_bn[0][0]']     
                                                                                                  
 block5c_add (Add)              (None, 5, 5, 112)    0           ['block5c_drop[0][0]',           
                                                                  'block5b_add[0][0]']            
                                                                                                  
 block6a_expand_conv (Conv2D)   (None, 5, 5, 672)    75264       ['block5c_add[0][0]']            
          

                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 3, 3, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 3, 3, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6c_expand_activation (Act  (None, 3, 3, 1152)  0           ['block6c_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6c_d

 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 3, 3, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 3, 3, 1152)  10368       ['block7a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block7a_bn (BatchNormalization  (None, 3, 3, 1152)  4608        ['block7a_dwconv[0][0]']         
 )                                                                                                
          

In [19]:
checkpoint = ModelCheckpoint(r'C:\Users\Admin\Documents\CS583_project\models\EfficientNetB0.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [23]:
custom_model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

custom_model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


C:\Users\Admin\AppData\Local\Temp\ipykernel_11900\4038297339.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  custom_model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


6367/6367 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5025
Epoch 1: val_loss did not improve from 0.69308
6367/6367 [==============================] - 844s 131ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4940 - lr: 0.0010
Epoch 2/5
6367/6367 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5023
Epoch 2: val_loss did not improve from 0.69308
6367/6367 [==============================] - 799s 126ms/step - loss: 0.6932 - accuracy: 0.5023 - val_loss: 0.6931 - val_accuracy: 0.5060 - lr: 0.0010
Epoch 3/5
6367/6367 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5009
Epoch 3: val_loss did not improve from 0.69308
6367/6367 [==============================] - 819s 129ms/step - loss: 0.6932 - accuracy: 0.5009 - val_loss: 0.6931 - val_accuracy: 0.5060 - lr: 0.0010
Epoch 4/5
6367/6367 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5026
Epoch 4: val_loss did not impro


print(acc_tr)
print(loss_tr)

In [7]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [29]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\Admin\Documents\CS583_project\models\inceptionv3.h5')

In [90]:
mixer.init()
sound= mixer.Sound(r'C:\Users\Admin\Documents\CS583_project\alarm.wav')
cap = cv2.VideoCapture(0)
Score = 0
while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    
    cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (255,0,0), thickness=3 )
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,0,0), thickness=3 )
        
        # preprocessing steps
        eye= frame[ey:ey+eh,ex:ex+w]
        eye= cv2.resize(eye,(80,80))
        eye= eye/255
        eye= eye.reshape(80,80,3)
        eye= np.expand_dims(eye,axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)
        
        # if eyes are closed
        if prediction[0][0]>0.20:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score=Score+1
            if(Score>1):
                try:
                    sound.play()
                except:
                    pass
            
        # if eyes are open
        elif prediction[0][1]>0.90:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score = Score-1
            if (Score<0):
                Score=0
            
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 385ms/step


KeyboardInterrupt: 